In [ ]:
from pymongo import MongoClient
import json

# This script is used to import a JSON file into MongoDB.
# I used similar code to upload most of my database collections.
# I used it to insert my collections: card_sets, pokemon_game, pokemon_information, pokemon_movies


client = MongoClient('mongodb://localhost:27017/')
db = client['pokemon_market_copy']  

with open('pokedex.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

collection = db['pokemon_information'] 
collection.insert_many(data)

print("Completed!")

In [ ]:
import os
import json

#since my pokemon_decks and card_informations are in 84 different files, so I used the code below to put them inside
#json file and upload into the json file later, I also separate my deck file into two collections pokemon_decks
#and deck_cards

deck_folder = 'decks'  
deck_output = []
deck_cards_output = []

for filename in os.listdir(deck_folder):
    if filename.endswith('.json'):
        filepath = os.path.join(deck_folder, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            decks = json.load(f)
            for deck in decks:
                deck_entry = {
                    "id": deck["id"],
                    "name": deck["name"],
                    "types": deck["types"]
                }
                deck_output.append(deck_entry)

                for card in deck["cards"]:
                    card_entry = {
                        "deck_id": deck["id"], 
                        "id": card.get("id", None), 
                        "name": card["name"],
                        "rarity": card.get("rarity", None),
                        "count": card["count"]
                    }
                    deck_cards_output.append(card_entry)

with open('decks.json', 'w', encoding='utf-8') as f:
    json.dump(deck_output, f, ensure_ascii=False, indent=2)

with open('deck_cards.json', 'w', encoding='utf-8') as f:
    json.dump(deck_cards_output, f, ensure_ascii=False, indent=2)

print("decks.json and deck_cards.json is successed")


In [ ]:
import os
import json
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['pokemon_market_copy']  

folder_path = '/mnt/data/your_folder' 

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, dict):
            if 'cards' in data:
                data = data['cards']
            else:
                data = [data]

        collection_name = filename.replace('.json', '') + '_cards'

        collection = db[collection_name]
        collection.insert_many(data)

print("Successful!")

In [ ]:
from pymongo import MongoClient
# This script adds a new "generation" field to each document in the "card_sets_cleaned" collection.
# It uses a predefined mapping from set IDs to Pokémon generations, so that we can later join 
# card sets with related game and movie data based on the generation.


client = MongoClient('mongodb://localhost:27017/')
db = client['pokemon_market_copy'] 
collection = db['card_sets']  

generation_mapping = {
    # Gen 1
    "base1": "Generation 1", "base2": "Generation 1", "base3": "Generation 1", 
    "base4": "Generation 1", "base5": "Generation 1", "base6": "Generation 1",
    "gym1": "Generation 1", "gym2": "Generation 1",

    # Gen 2
    "neo1": "Generation 2", "neo2": "Generation 2", "neo3": "Generation 2", "neo4": "Generation 2",
    "ecard1": "Generation 2", "ecard2": "Generation 2", "ecard3": "Generation 2",

    # Gen 3
    "ex1": "Generation 3", "ex2": "Generation 3", "ex3": "Generation 3", "ex4": "Generation 3",
    "ex5": "Generation 3", "ex6": "Generation 3", "ex7": "Generation 3", "ex8": "Generation 3",
    "ex9": "Generation 3", "ex10": "Generation 3", "ex11": "Generation 3", "ex12": "Generation 3",
    "ex13": "Generation 3", "ex14": "Generation 3", "ex15": "Generation 3", "ex16": "Generation 3",
    "pop1": "Generation 3", "pop2": "Generation 3", "pop3": "Generation 3", "pop4": "Generation 3", "pop5": "Generation 3",

    # Gen 4
    "dp1": "Generation 4", "dp2": "Generation 4", "dp3": "Generation 4", "dp4": "Generation 4",
    "dp5": "Generation 4", "dp6": "Generation 4", "dp7": "Generation 4",
    "pl1": "Generation 4", "pl2": "Generation 4", "pl3": "Generation 4", "pl4": "Generation 4",
    "hgss1": "Generation 4", "hgss2": "Generation 4", "hgss3": "Generation 4", "hgss4": "Generation 4",
    "pop6": "Generation 4", "pop7": "Generation 4", "pop8": "Generation 4", "pop9": "Generation 4",

    # Gen 5
    "bw1": "Generation 5", "bw2": "Generation 5", "bw3": "Generation 5", "bw4": "Generation 5",
    "bw5": "Generation 5", "bw6": "Generation 5", "bw7": "Generation 5", "bw8": "Generation 5",
    "bw9": "Generation 5", "bw10": "Generation 5", "bw11": "Generation 5",

    # Gen 6
    "xy0": "Generation 6", "xy1": "Generation 6", "xy2": "Generation 6", "xy3": "Generation 6",
    "xy4": "Generation 6", "xy5": "Generation 6", "xy6": "Generation 6", "xy7": "Generation 6",
    "xy8": "Generation 6", "xy9": "Generation 6", "xy10": "Generation 6", "xy11": "Generation 6", "xy12": "Generation 6",

    # Gen 7
    "sm1": "Generation 7", "sm2": "Generation 7", "sm3": "Generation 7", "sm4": "Generation 7",
    "sm5": "Generation 7", "sm6": "Generation 7", "sm7": "Generation 7", "sm8": "Generation 7",
    "sm9": "Generation 7", "sm10": "Generation 7", "sm11": "Generation 7", "sm12": "Generation 7",
    "sm35": "Generation 7", "sm75": "Generation 7", "sm115": "Generation 7",

    # Gen 8
    "swsh1": "Generation 8", "swsh2": "Generation 8", "swsh3": "Generation 8", "swsh4": "Generation 8",
    "swsh5": "Generation 8", "swsh6": "Generation 8", "swsh7": "Generation 8", "swsh8": "Generation 8",
    "swsh9": "Generation 8", "swsh10": "Generation 8", "swsh11": "Generation 8", "swsh12": "Generation 8",
    "swsh35": "Generation 8", "swsh45": "Generation 8", "swsh45sv": "Generation 8",

    # Gen 9
    "sv1": "Generation 9", "sv2": "Generation 9", "sv3": "Generation 9", "sv3pt5": "Generation 9",
    "sv4": "Generation 9", "sv4pt5": "Generation 9", "sv5": "Generation 9",
    "sv6": "Generation 9", "sv6pt5": "Generation 9", "sv7": "Generation 9",
    "sv8": "Generation 9", "sv8pt5": "Generation 9",
}

for doc in collection.find():
    set_id = doc.get("id", "")
    gen = generation_mapping.get(set_id, "Unknown Generation") 
    collection.update_one({"_id": doc["_id"]}, {"$set": {"generation": gen}})

print("Successful!")


In [ ]:
import requests
import urllib3
from bs4 import BeautifulSoup
import time
from pymongo import MongoClient
import re
import random

# This script collects the most recent eBay listing price for each Pokémon card
# I skip cards with "pop" in the name (graded/promotional cards) and process in reverse to balance request timing

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_price_from_ebay(search_term):
    url = "https://www.ebay.com/sch/i.html"
    params = {"_nkw": search_term, "_ipg": "50"}
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        res = requests.get(url, params=params, headers=headers, verify=False, timeout=10)
        if res.status_code != 200:
            print(f"Failed to fetch {search_term} — HTTP {res.status_code}")
            return None
    except Exception as err:
        print(f"Request error for {search_term}: {err}")
        return None

    soup = BeautifulSoup(res.text, "html.parser")
    for item in soup.select(".s-item"):
        title_elem = item.select_one(".s-item__title")
        price_elem = item.select_one(".s-item__price")

        if not title_elem or not price_elem:
            continue

        title = title_elem.text.strip()
        if "Shop on eBay" in title or "Explore" in title:
            continue

        price = price_elem.text.strip()

        return {
            "search_name": search_term,
            "title": title,
            "price": price
        }

    return None

def save_price_to_db(price_doc, db_name="pokemon_market_copy", collection_name="card_price"):
    mongo = MongoClient("mongodb://localhost:27017/")
    db = mongo[db_name]
    db[collection_name].insert_one(price_doc)

if __name__ == "__main__":
    mongo = MongoClient("mongodb://localhost:27017/")
    db = mongo["pokemon_market_copy"]
    cards = list(db["card_information"].find(
        {"search_name": {"$not": re.compile("pop", re.IGNORECASE)}},
        {"search_name": 1, "_id": 0}
    ))

    total = len(cards)
    start_from = 11248  # This is where I last left off scraping

    try:
        for offset, card in enumerate(reversed(cards[:start_from + 1])):
            i = start_from - offset
            keyword = card["search_name"]

            listing = fetch_price_from_ebay(keyword)
            if listing:
                save_price_to_db(listing)

            time.sleep(random.uniform(1, 2))  # Delay to avoid getting banned

    except Exception as e:
        print(f"Error occurred at card #{i}")
        print(f"Details: {e}")

    finally:
        print(f"Finished up to card #{i}")
